# Solutions to in-class assignment

Note that **the current notebook does not yet have the solutions implemented**!!!!!!!  I just copied this over in Spring 2021 but didn't get to implement it.  Set it up at some point!


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def rk4(y1_0,y2_0,rhs,xl=0.0,xr=1.0,n=100):
    '''
    does 4th order Runge-Kutta for given y1(xl), y2(xl), rhs (right hand side 
    of equation to be integrated) integrates to xy over n steps.  Assume that
    we only have 2 equations to make the code clearer, though note that this whole
    set of solutions can in principle be written for a vector of equations rather than
    two if we wanted to do so.
    '''

    # set up our arrays and step size
    y1 = np.zeros(n)
    y2 = np.zeros(n)
    x = np.linspace(xl, xr, n)
    h = x[1] - x[0]  # stepsize

    # set up left boundary
    y1[0] = y1_0
    y2[0] = y2_0

    # integrate from left to right boundary using given rhs function.
    for m in range(n-1):
        dy1dx_1, dy2dx_1 = rhs(y1[m], y2[m])
        dy1dx_2, dy2dx_2 = rhs(y1[m] + 0.5*h*dy1dx_1, y2[m] + 0.5*h*dy2dx_1)
        dy1dx_3, dy2dx_3 = rhs(y1[m] + 0.5*h*dy1dx_2, y2[m] + 0.5*h*dy2dx_2)
        dy1dx_4, dy2dx_4 = rhs(y1[m] + h*dy1dx_3, y2[m] + h*dy2dx_3)

        y1[m+1] = y1[m] + (h/6.0)*(dy1dx_1 + 2.0*dy1dx_2 + 2.0*dy1dx_3 + dy1dx_4)
        y2[m+1] = y2[m] + (h/6.0)*(dy2dx_1 + 2.0*dy2dx_2 + 2.0*dy2dx_3 + dy2dx_4)

    return y1, y2  # 2 arrays


def rhs(y1, y2):
    '''
    right hand side (i.e., derivatives) from pre-class assignment.
    Original equation is y'' = 4y
    Linearized equations are:
    dy1/dx = y2
    dy2/dx = 4y1
    
    Note that y1 = y
    '''
    dy1dx = y2
    dy2dx = 4.0*y1

    return dy1dx, dy2dx


def analytic_f(x):
    # analytic function we're trying to match
    return 2.0*np.exp(2.0*x) + 3.0*np.exp(-2.0*x)

def analytic_fprime(x):
    # analytic derivative we're trying to match (just for fun)
    return 4.0*np.exp(2.0*x) - 6.0*np.exp(-2.0*x)


In [ ]:
# boundaries for equation
x_left = 0.0
x_right = 2.0

# these are the values we actually know are true
y1_left_true = 5.0
y2_right_true = 218.282706

# this many steps in interval (doesn't really matter)
points=20

# desired accuracy
eps = 1.0e-6



In [ ]:
# set this to be large
dy = 1.0e+4*eps

y1_0 = y1_left_true  # correct
y2_0 = 0.0  # will adjust

# make a first guess to get some test values for y1, y2 
# (as arrays for plotting convenience - we really only 
#  care about the last points, though!)
y1_old, y2_old = rk4(y1_0,y2_0,rhs,xl=x_left,xr=x_right,n=points)

y2_tm1 = y2_0  # y2[0] at t-1
y2_0 = 0.1    # new guess for y2[0] - will get modified

iter = 0

'''
Iterate using secant method: we want to get the correct value of 
eta, which is y2(0) - the parameter we're trying to find that gives 
us the correct value of y2(2).  So, we want to zero out the difference 
between our estimate of y2(2) and our known value of y2(2), or more
accurately zero:

f(eta) = y2^eta(2) - y2_true(2)

df/deta is the change in y2(2) as we change our estimates for y2(0).
'''
while dy > eps and iter < 10: 
    
    # get new values of y1 and y2 arrays
    y1, y2 = rk4(y1_0,y2_0,rhs,xl=x_left,xr=x_right,n=points)
    
    # technically this is ((y2[p-1]-y2_r_true)-(y2_old[p-1]-y2_r_true))/(y2_0-y2_tml)
    # but I simplified it for ease of reading.
    dfdeta = (y2[points-1] - y2_old[points-1])/(y2_0-y2_tm1)

    deta = -(y2[points-1]-y2_right_true)/dfdeta

    
    y2_tm1 = y2_0
    y2_0 += deta
    
    dy = abs(deta)
    iter += 1
    
    y1_old = y1
    y2_old = y2
    
    print(y1[0],y2[0],iter,dy)
    

x = np.linspace(x_left,x_right,points)
y_an = analytic_f(x)

plt.plot(x,y1,'r-',x,y_an,'b.')

Now solve using Scipy's [integrate.solve_bvp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_bvp.html) method.